# Metawrap stool metagenomes from TEDDY  
This is in a temporary folder because /projects/teddy/ is not yet showing up on Jupyter Hub

In [1]:
import pandas as pd
from glob import glob
from collections import defaultdict
import os
import pandas as pd

In [2]:
md = pd.read_csv("/projects/teddy/20190716_TEDDY_Cleaned_Metadata_v2.tsv", sep='\t', index_col=0)

/opt/miniconda3/envs/jupyter_qiime/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (38,39,42,60,62,64,68,74,78,82,84,86,88,90,92,94,96,100,102,106,108,110,112,114,116,124,126,128,130,132,134,136,138,140,142,144,146,150,151,152,153,159,165,166,167,168,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,187,189,381,382,383,386,387,388,389,390,391,392,393,396,397,398,401,402,403,404,405,406,407,408,409) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Make lists of finished/unfinished hosts

In [3]:
##Make map of sample_mask to host_subject_id
host_id_dict = {row.sample_mask_id:row.host_subject_id for _id,row in md.iterrows()}
host_id_df = pd.DataFrame.from_dict(host_id_dict, orient="index").sort_index()
all_hosts = set(value for key,value in host_id_dict.items())
#host_id_df.to_csv("host_id_map.tsv", sep='\t', header=None)

#Make list of uniqe host_ids
! [ ! -f host_ids.txt ] && cut -f25 /projects/teddy/20190716_TEDDY_Cleaned_Metadata_v2.tsv | tail -n +2  | uniq > host_ids.txt

###Get/import list of hosts with sequencing data
host_with_data_file="hosts_with_data.txt"
empty_hosts_file="empty_hosts.txt"
hosts_with_data=set()
empty_hosts=set()
if os.path.exists(host_with_data_file):
    with open(host_with_data_file,'r') as openfile:
        for line in openfile:
            hosts_with_data.add(line.strip())
    with open(empty_hosts_file,'r') as openfile:
        for line in openfile:
            empty_hosts.add(line.strip())    
else:
    empty, some = 0, 0
    temp_d = defaultdict(list)
    #Get list of hosts
    for _id,row in md.iterrows():
        temp_d[row.host_subject_id].append(row.sample_mask_id)
    for host,sample_list in temp_d.items():
        files = []
        for s in sample_list:
            files.extend(glob("/projects/teddy/STOOL-WGS-V3/*/" + str(s) + "/processed/*.fastq.bz2"))
            files.extend(glob("/projects/teddy/STOOL-WGS-V3/*/*/" + str(s) + "/processed/*.fastq.bz2"))
        if not files:
            empty += 1
            empty_hosts.add(host)
        else:
            some +=1
            hosts_with_data.add(host)
    #Write output
    with open(host_with_data_file,'w') as openfile:
        for h in hosts_with_data:
            openfile.write(h + '\n')
    with open(empty_hosts_file,'w') as openfile:
        for h in empty_hosts:
            openfile.write(h + '\n')

#Get finished hosts
finished_hosts = set()
with open("successful_assemblies.txt",'r') as openfile:
    for line in openfile:
        finished_hosts.add(line.strip())
        
#Get failed hosts
failed_hosts = set()
with open("failed_assemblies.txt",'r') as openfile:
    for line in openfile:
        failed_hosts.add(line.strip())

#Get unfinished hosts and save to txt file
unfinished_hosts = hosts_with_data - finished_hosts - failed_hosts
with open("unfinished_hosts.txt", 'w') as openfile:
     for h in unfinished_hosts:
            openfile.write(h + '\n')


print("Hosts with data: {}".format(len(hosts_with_data)))
print("Hosts with no data: {}".format(len(empty_hosts)))
print("Finished hosts: {}".format(len(finished_hosts)))
print("Unfinished hosts: {}".format(len(unfinished_hosts)))
print("Failed hosts: {}".format(len(failed_hosts)))

Hosts with data: 858
Hosts with no data: 24
Finished hosts: 731
Unfinished hosts: 79
Failed hosts: 49


In [4]:
%%script bash
echo '
#!/bin/bash -l
#PBS -S /bin/bash
#PBS -e /projects/teddy/metawrap/logs
#PBS -o /projects/teddy/metawrap/logs
#PBS -l walltime=48:00:00
#PBS -l nodes=1:ppn=12
#PBS -l mem=128gb
#PBS -N teddy_metawrap_assemble

source ~/.bash_profile
conda activate metawrap-env

export TMPDIR=/panfs/panfs1.ucsd.edu/panscratch/$USER
tmp=$(mktemp -d --tmpdir)
export TMPDIR=$tmp
trap "rm -r $tmp; unset TMPDIR" EXIT
cd $tmp

host_id=$( sed -n "$PBS_ARRAYID"p /projects/teddy/metawrap/unfinished_hosts.txt )
    
out_dir=/projects/teddy/metawrap/assemblies/
success_file=/projects/teddy/metawrap/successful_assemblies.txt
failure_file=/projects/teddy/metawrap/failed_assemblies.txt
#Search for all samples within host_id
grep $host_id /projects/teddy/metawrap/host_id_map.tsv | cut -f1 |\
#For each sample, concatenate reads into one file in tmp
while read sample_id
    do
    echo $sample_id
    find /projects/teddy/STOOL-WGS-V3/ -name "$sample_id" |\
    while read folder
        do
        cat $folder/processed/*1.fastq.bz2 >> $tmp/$host_id.r1.fastq.bz2
        cat $folder/processed/*2.fastq.bz2 >> $tmp/$host_id.r2.fastq.bz2
        done
    done
    #Unzip
    bzip2 -d $tmp/$host_id.r1.fastq.bz2
    bzip2 -d $tmp/$host_id.r2.fastq.bz2
#Run metawrap on merged fastq of host_id
metawrap assembly \
    --metaspades \
    -1 $tmp/$host_id.r1.fastq \
    -2 $tmp/$host_id.r2.fastq \
    -o $tmp/$host_id \
    -m 128 \
    -t 8 \
    -l 1000
#Cleanup
#Check if completed successfully
if [ -f $tmp/$host_id/final_assembly.fasta ]
    then
    echo $host_id >> $success_file
    #Delete intermediate files
    for int_folder in {K21,K33,K55,corrected,misc}
        do
        [ -d  $tmp/$host_id/metaspades/$int_folder ] && rm -r $tmp/$host_id/metaspades/$int_folder
        done
    #Compress metaspades folder
    tar cf - $tmp/$host_id/metaspades/ | xz -z -1 - > $tmp/$host_id/metaspades.tar.xz
    #Delete uncompressed folder
    rm -r $tmp/$host_id/metaspades/
    #Move to final destination
    mv $tmp/$host_id $out_dir/
    #Report if failed
    else
        echo $host_id >> $failure_file
    fi


    
' | qsub -t 1-113%30

1355090[].barnacle.ucsd.edu


## Clean up metaspades folders

In [2]:
%%script bash

out_dir=/projects/teddy/metawrap/assemblies

for folder in $out_dir/pt*
    do
    echo $folder
    for int_folder in {K21,K33,K55,corrected,misc}
        do
        [ -d $folder/metaspades/$int_folder ] && rm -r $folder/metaspades/$int_folder
        done
    done


Process is interrupted.


# Binning

### Make list of unfinished binning

In [4]:
host_with_data_file="successful_assemblies.txt"
hosts_with_data=set()
with open(host_with_data_file,'r') as openfile:
    for line in openfile:
        hosts_with_data.add(line.strip())
            
#Get finished hosts
finished_bins = set()
with open("successful_bins.txt",'r') as openfile:
    for line in openfile:
        finished_bins.add(line.strip())
        
#Get failed hosts
failed_bins = set()
with open("failed_bins.txt",'r') as openfile:
    for line in openfile:
        failed_bins.add(line.strip())

#Get unfinished hosts and save to txt file
unfinished_bins = hosts_with_data - finished_bins - failed_bins
with open("unfinished_bins.txt", 'w') as openfile:
     for h in unfinished_bins:
            openfile.write(h + '\n')
            
print("Finished bins: {}".format(len(finished_bins)))
print("Unfinished bins: {}".format(len(unfinished_bins)))
print("Failed bins: {}".format(len(failed_bins)))

Finished bins: 656
Unfinished bins: 56
Failed bins: 3


In [12]:
pwd

'/projects/teddy/metawrap'

Binning takes < 10 Gig  
Bin_refinement takes a while because of checkM I think.  
Without --quick on bin_refinement, takes > 145 Gb. With --quick, takes 14Gb

In [5]:
%%script bash
echo '
#!/bin/bash -l
#PBS -S /bin/bash
#PBS -M swandro@ucsd.edu
#PBS -e /projects/teddy/metawrap/logs
#PBS -o /projects/teddy/metawrap/logs
#PBS -l walltime=8:00:00
#PBS -l nodes=1:ppn=10
#PBS -l mem=32gb
#PBS -N bin_teddy

source ~/.bash_profile
conda activate metawrap-env

export TMPDIR=/panfs/panfs1.ucsd.edu/panscratch/$USER
tmp=$(mktemp -d --tmpdir)
export TMPDIR=$tmp
trap "rm -r $tmp; unset TMPDIR" EXIT
cd $tmp

host_id=$( sed -n "$PBS_ARRAYID"p /projects/teddy/metawrap/unfinished_bins.txt )
    
out_dir=/projects/teddy/metawrap/binning/
mkdir -p $out_dir
success_file=/projects/teddy/metawrap/successful_bins.txt
failure_file=/projects/teddy/metawrap/failed_bins.txt
log_dir=/projects/teddy/metawrap/binning_logs/

#Search for all samples within host_id
grep $host_id /projects/teddy/metawrap/host_id_map.tsv | cut -f1 |\
#For each sample, concatenate reads into one file in tmp
while read sample_id
    do
    echo $sample_id
    find /projects/teddy/STOOL-WGS-V3/ -name "$sample_id" |\
    while read folder
        do
        cat $folder/processed/*1.fastq.bz2 >> $tmp/$host_id._1.fastq.bz2
        cat $folder/processed/*2.fastq.bz2 >> $tmp/$host_id._2.fastq.bz2
        done
    done
    #Unzip
    bzip2 -d $tmp/$host_id._1.fastq.bz2
    bzip2 -d $tmp/$host_id._2.fastq.bz2
    
    
#Run metawrap binning on merged fastq of host_id
start=`date +%s`
    /usr/bin/time -v -o $log_dir/individual_binning/${host_id}.log.txt -a -- metawrap binning \
        --maxbin2 \
        --metabat2 \
        -a /projects/teddy/metawrap/assemblies/$host_id/final_assembly.fasta \
        -o $tmp/${host_id}_bins \
        -m 32 \
        -t 10 \
        -l 1000 \
    $tmp/$host_id._1.fastq \
    $tmp/$host_id._2.fastq
    mv $tmp/$host_id $out_dir/

#Run bin refinement
/usr/bin/time -v -o $log_dir/individual_bin_refinement/${host_id}.log.txt -a -- metawrap bin_refinement \
        --quick \
        -m 32 \
        -t 10 \
        -A $tmp/${host_id}_bins/maxbin2_bins \
        -B $tmp/${host_id}_bins/metabat2_bins \
        -o $tmp/${host_id}


#If successful
if [ -f $tmp/${host_id}/metawrap_70_10_bins.contigs ]
    then
    end=`date +%s`
    runtime=$((end-start))
    echo "Finished bin refinement ${host_id} in $runtime seconds." >> $log_dir/bin_time_log.txt
    echo $host_id >> $success_file
    rm -r $tmp/${host_id}/work_files/
    mv $tmp/${host_id} $out_dir
    
    else
    echo $host_id >> $failure_file
fi
    
' | qsub -t 1-56%20

1357983[].barnacle.ucsd.edu


# bin_refinement
Not doing

In [ ]:
test_folder=/projects/teddy/metawrap/binning/pt997004/maxbin2_bins

In [44]:
%%script bash

echo '
from sys import argv
import pandas as pd

sample_dir=argv[1]

#Import checkM file
checkm_file = pd.read_csv( sample_dir + "/maxbin2_bins.stats", sep="\t")
#Get list of incomplete genomes
incomplete_genomes = list( checkm_file.query("completeness < 70 | contamination > 10").bin )
complete_genomes = list( checkm_file.query("completeness > 70 & contamination < 10").bin )

with open(sample_dir + "/maxbin2_70_10_bins.txt", "w") as openfile:
    for bin_ in complete_genomes:
        openfile.write(bin_ + "\n")
with open(sample_dir + "/maxbin2_not_70_10_bins.txt", "w") as openfile:
    for bin_ in incomplete_genomes:
        openfile.write(bin_ + "\n")
' > /projects/teddy/metawrap/utils/get_bins.py


Creates files:  
- maxbin2_70_10_bins.txt
- maxbin2_not_70_10_bins.txt

In [45]:
%%script bash

for folder in /projects/teddy/metawrap/binning/pt*;
    do
    if [ ! -d $folder/maxbin_70_10_bins ];
        then
        python /projects/teddy/metawrap/utils/get_bins.py $folder

        mkdir $folder/maxbin_70_10_bins
        while read line; do 
            cp $folder/maxbin2_bins/$line.fa  $folder/maxbin_70_10_bins
        done < $folder/maxbin2_70_10_bins.txt 

        while read line; do 
            cat $folder/maxbin2_bins/$line.fa >> $folder/maxbin_70_10_bins/lowQ_bins.fa
        done < $folder/maxbin2_not_70_10_bins.txt
        fi
    done

Traceback (most recent call last):
  File "/projects/teddy/metawrap/utils/get_bins.py", line 8, in <module>
    checkm_file = pd.read_csv( sample_dir + "/maxbin2_bins.stats", sep="\t")
  File "/home/swandro/miniconda3/lib/python3.7/site-packages/pandas/io/parsers.py", line 685, in parser_f
    return _read(filepath_or_buffer, kwds)
  File "/home/swandro/miniconda3/lib/python3.7/site-packages/pandas/io/parsers.py", line 457, in _read
    parser = TextFileReader(fp_or_buf, **kwds)
  File "/home/swandro/miniconda3/lib/python3.7/site-packages/pandas/io/parsers.py", line 895, in __init__
    self._make_engine(self.engine)
  File "/home/swandro/miniconda3/lib/python3.7/site-packages/pandas/io/parsers.py", line 1135, in _make_engine
    self._engine = CParserWrapper(self.f, **self.options)
  File "/home/swandro/miniconda3/lib/python3.7/site-packages/pandas/io/parsers.py", line 1917, in __init__
    self._reader = parsers.TextReader(src, **kwds)
  File "pandas/_libs/parsers.pyx", line 382, in 

Creates folder:
- maxbin_70_10_bins

# Prokka

In [5]:
%%script bash
echo '
#!/bin/bash -l
#PBS -S /bin/bash
#PBS -M swandro@ucsd.edu
#PBS -e /projects/teddy/metawrap/logs
#PBS -o /projects/teddy/metawrap/logs
#PBS -l walltime=72:00:00
#PBS -l nodes=1:ppn=10
#PBS -l mem=16gb
#PBS -N teddy_prokka

source ~/.bash_profile
conda activate biopython

export TMPDIR=/panfs/panfs1.ucsd.edu/panscratch/$USER
tmp=$(mktemp -d --tmpdir)
export TMPDIR=$tmp
trap "rm -r $tmp; unset TMPDIR" EXIT
cd $tmp


for folder in /projects/teddy/metawrap/binning/pt*
    do
    bin_dir=$folder/maxbin_70_10_bins
    out_root=$bin_dir/prokka_annotation
    #Make prokka_annotation dir
    
    #Iterate through bins and run prokka
    for file in $bin_dir/*.fa
        do
        #Get name
        no_root=${file##*/}
        base_name=${no_root%.fa}
        #If .gbk file doesnt exist
        if [ ! -f $out_root/$base_name/$base_name.gbk ]
            then
            mkdir -p $out_root/$base_name
            #set output dir in tmp
            out_dir=$tmp/$base_name
            #Run prokka
            prokka --cpus 10 \
                      --compliant \
                      --outdir $out_dir \
                      --prefix $base_name \
                      $file
            #Move to destination
            mv $out_dir $out_root
            fi
        done
    done
' | qsub

1363282.barnacle.ucsd.edu


# Bin quantification

In [109]:
host_with_data_file="successful_assemblies.txt"
hosts_with_data=set()
with open(host_with_data_file,'r') as openfile:
    for line in openfile:
        hosts_with_data.add(line.strip())
            
#Get finished hosts
finished_bins = set()
with open("bin_quant_logs/successful_bin_quant.txt",'r') as openfile:
    for line in openfile:
        finished_bins.add(line.strip())
        
#Get failed hosts
failed_bins = set()
with open("bin_quant_logs/failed_bin_quant.txt",'r') as openfile:
    for line in openfile:
        failed_bins.add(line.strip())

#Get unfinished hosts and save to txt file
unfinished_bins = hosts_with_data - finished_bins - failed_bins
with open("bin_quant_logs/unfinished_bin_quant.txt", 'w') as openfile:
     for h in unfinished_bins:
            openfile.write(h + '\n')
            
print("Finished bin quant: {}".format(len(finished_bins)))
print("Unfinished bin quant: {}".format(len(unfinished_bins)))
print("Failed bin quant: {}".format(len(failed_bins)))

Finished bin quant: 107
Unfinished bin quant: 610
Failed bin quant: 0


In [110]:
%%script bash
echo '
#!/bin/bash -l
#PBS -S /bin/bash
#PBS -M swandro@ucsd.edu
#PBS -e /projects/teddy/metawrap/logs
#PBS -o /projects/teddy/metawrap/logs
#PBS -l walltime=4:00:00
#PBS -l nodes=1:ppn=4
#PBS -l mem=16gb
#PBS -N quant_bins_teddy

source ~/.bash_profile
conda activate metawrap-env

export TMPDIR=/panfs/panfs1.ucsd.edu/panscratch/$USER
tmp=$(mktemp -d --tmpdir)
export TMPDIR=$tmp
trap "rm -r $tmp; unset TMPDIR" EXIT
cd $tmp

host_id=$( sed -n "$PBS_ARRAYID"p /projects/teddy/metawrap/bin_quant_logs/unfinished_bin_quant.txt )
    

log_dir=/projects/teddy/metawrap/bin_quant_logs/
success_file=$log_dir/successful_bin_quant.txt
failure_file=$log_dir/failed_bin_quant.txt
individual_log_dir=$log_dir/individual_logs
time_log_file=$log_dir/bin_quant_time_log.txt

#Search for all samples within host_id
grep $host_id /projects/teddy/metawrap/host_id_map.tsv | cut -f1 |\
#For each sample, concatenate reads into one file in tmp
while read sample_id
    do
    echo $sample_id
    find /projects/teddy/STOOL-WGS-V3/ -name "$sample_id" |\
    while read folder
        do
        cat $folder/processed/*1.fastq.bz2 >> $tmp/$host_id._1.fastq.bz2
        cat $folder/processed/*2.fastq.bz2 >> $tmp/$host_id._2.fastq.bz2
        done
    done
    #Unzip
    bzip2 -d $tmp/$host_id._1.fastq.bz2
    bzip2 -d $tmp/$host_id._2.fastq.bz2

bin_dir=/projects/teddy/metawrap/binning/$host_id/maxbin_70_10_bins
out_tmp=$tmp/maxbin_quant
out_dir=/projects/teddy/metawrap/binning/$host_id/

start=`date +%s`
/usr/bin/time -v -o $individual_log_dir/${host_id}.log.txt -a -- metawrap quant_bins \
    -b $bin_dir \
    -o $out_tmp \
    -a /projects/teddy/metawrap/assemblies/$host_id/final_assembly.fasta \
    -t 4 \
    $tmp/$host_id._1.fastq $tmp/$host_id._2.fastq
    
if [ -f $out_tmp/bin_abundance_table.tab ]
    then
    #Delete the assembly index
    rm -r $out_tmp/assembly_index
    #Move from temp to final destination
    mv $out_tmp $out_dir
    #Record success
    echo $host_id >> $success_file
    end=`date +%s`
    runtime=$((end-start))
    echo "Finished bin quantification ${host_id} in $runtime seconds." >> $time_log_file
    
    else
    echo $host_id >> $failure_file
    fi
    
' | qsub -t 1-610%20

1359206[].barnacle.ucsd.edu


# Bin classification

Metawrap needs nt unzipped. Barnalce only has nt zipped.

In [44]:
%%script bash
echo '
#!/bin/bash -l
#PBS -S /bin/bash
#PBS -M swandro@ucsd.edu
#PBS -e /projects/teddy/metawrap/bin_id_logs/logs
#PBS -o /projects/teddy/metawrap/bin_id_logs/logs
#PBS -l walltime=24:00:00
#PBS -l nodes=1:ppn=4
#PBS -l mem=32gb
#PBS -N t_bin_classification

source ~/.bash_profile
conda activate metawrap-env

export TMPDIR=/panfs/panfs1.ucsd.edu/panscratch/$USER
tmp=$(mktemp -d --tmpdir)
export TMPDIR=$tmp
trap "rm -r $tmp; unset TMPDIR" EXIT
cd $tmp

test_bin_folder=/projects/teddy/metawrap/binning/pt212774/maxbin_70_10_bins
out_dir=/projects/teddy/metawrap/CAT_BAT/test_bin_metawrap

cp $test_bin_folder/bin*.fa $tmp

mkdir $out_dir

start=`date +%s`
/usr/bin/time -v -o $out_dir/memory_log.txt -a -- metaWRAP classify_bins \
    -b $tmp \
    -t 4 \
    -o $out_dir
end=`date +%s`
runtime=$((end-start))
echo "Finished metawrap bin calssification in $runtime seconds." >> $out_dir/time_log.txt


' | qsub

1359580.barnacle.ucsd.edu


# Calculate sequencing depth per patient

# Make file lists for each individual

In [46]:
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles


#Make sample id map
sample_id_map = {x.split()[0]:x.split()[1] for x in open("/projects/teddy/metawrap/host_id_map.tsv").readlines()}

#Get all files
all_files = getListOfFiles('/projects/teddy/STOOL-WGS-V3')

#Make lists of files
file_path_by_host = defaultdict(list)
files_without_md=[]
for file in all_files:
    splt = file.split('/')
    try:
        host_id=sample_id_map[splt[-3]]
        file_path_by_host[host_id].append(file)
    except:
        files_without_md.append(file)
        
#Write files
for host_id,file_list in file_path_by_host.items():
    with open("/projects/teddy/metawrap/file_lists_by_host/{}.txt".format(host_id), 'w') as outfile:
        f_files=sorted([x for x in file_list if "1.fastq.bz2" in x])
        r_files=sorted([x for x in file_list if "2.fastq.bz2" in x])
        for f_file, r_file in zip(f_files, r_files):
            outfile.write("{}\t{}\n".format(f_file, r_file))

File Sizes

In [96]:
size_dict={}

for host_id, file_list in file_path_by_host.items():
    tot_size=0
    for file in file_list:
        tot_size += os.stat(file).st_size
    size_dict[host_id] = tot_size
    
size_df = pd.DataFrame.from_dict(size_dict, orient="index")
size_df.columns=["bytes"]
size_df.index.name="host_subject_id"
size_df.to_csv("/projects/teddy/metawrap/file_size_by_host.tsv", sep='\t')

In [45]:
size_df = pd.read_csv("/projects/teddy/metawrap/file_size_by_host.tsv", sep='\t')

In [50]:
size_df.bytes.sum()/1000000000

10933.689453793

In [12]:
%%script bash
echo '
#!/bin/bash -l
#PBS -S /bin/bash
#PBS -M swandro@ucsd.edu
#PBS -e /projects/teddy/metawrap/logs
#PBS -o /projects/teddy/metawrap/logs
#PBS -l walltime=24:00:00
#PBS -l nodes=1:ppn=2
#PBS -l mem=16gb
#PBS -N count_teddy

sample_number_file=/projects/teddy/metawrap/sample_info/sample_numbers.txt
read_count_file=/projects/teddy/metawrap/sample_info/by_sample/

total_lines=0
total_samples=0
cat /projects/teddy/metawrap/hosts_with_data.txt |\
while read host_id
    do
    #Search for all samples within host_id
    grep $host_id /projects/teddy/metawrap/host_id_map.tsv | cut -f1 |\
    #For each sample, concatenate reads into one file in tmp
    while read sample_id
        do
        echo $sample_id
        find /projects/teddy/STOOL-WGS-V3/ -name "$sample_id" |\
        while read folder
            do
            for file in $folder/processed/*1.fastq.bz2
                do
                ll 
                total_lines=$(($total_lines+(`bzcat $file`/4)))
                total_samples=$(($total_samples+1))
                done
            done
        done
    echo "${host_id}\t${total_samples}" >> $sample_number_file
    echo "${host_id}\t${total_lines}" >> $read_count_file
    done
' | qsub

1359195.barnacle.ucsd.edu


In [8]:
%%script bash

sample_number_file=/projects/teddy/metawrap/sample_info/sample_numbers.txt
read_count_file=/projects/teddy/metawrap/sample_info/read_counts.txt

total_lines=0
total_samples=0
cat /projects/teddy/metawrap/hosts_with_data.txt |\
while read host_id
    do
    echo $host_id
    #Search for all samples within host_id
    grep $host_id /projects/teddy/metawrap/host_id_map.tsv | cut -f1 |\
    #For each sample, concatenate reads into one file in tmp
    while read sample_id
        do
        echo $sample_id
        find /projects/teddy/STOOL-WGS-V3/ -name "$sample_id" |\
        while read folder
            do
            for file in $folder/processed/*1.fastq.bz2
                do
                total_samples=$(($total_samples+1))
                done
            done
        done
    echo "${host_id}\t${total_samples}"
    done

Process is interrupted.


# Make file size list for each individual

In [15]:
%%script bash
echo '
#!/bin/bash -l
#PBS -S /bin/bash
#PBS -M swandro@ucsd.edu
#PBS -e /projects/teddy/metawrap/logs
#PBS -o /projects/teddy/metawrap/logs
#PBS -l walltime=24:00:00
#PBS -l nodes=1:ppn=2
#PBS -l mem=16gb
#PBS -N count_teddy

source ~/.bash_profile
read_count_folder=/projects/teddy/metawrap/sample_info/by_sample/
cat /projects/teddy/metawrap/hosts_with_data.txt |\
while read host_id
    do
    #Search for all samples within host_id
    grep $host_id /projects/teddy/metawrap/host_id_map.tsv | cut -f1 |\
    #For each sample, concatenate reads into one file in tmp
    while read sample_id
        do
        find /projects/teddy/STOOL-WGS-V3/ -name "$sample_id" |\
        while read folder
            do
            ll $folder >> $read_count_folder/host_id
            done
        done
    done
' | qsub

1359196.barnacle.ucsd.edu
